In [22]:
import csv
import pickle

content = []

with open('ftse_100_time_data.csv', newline='') as f:
    reader = csv.reader(f)
    for row in reader:
        content.append(row)


data_ftse_100_dict = {}
tickers_list = [ticker[:-2] for ticker in content[0][1:]]


for row in content[1:]:
    data_ftse_100_dict[row[0]] = {}
    for i, ticker in enumerate(tickers_list):
        data_ftse_100_dict[row[0]][ticker] = row[i]

pickle.dump(data_ftse_100_dict, open("./data_ftse_100_dict_04032017.p", "wb"))


In [21]:
data_ftse_100_dict['1990-07-12']

{'AAL': '1990-07-12',
 'ABF': '',
 'ADM': '363.44',
 'ADN': '',
 'AGK': '',
 'AHT': '',
 'ANTO': '197.584',
 'ARM': '600.21',
 'AV': '',
 'AZN': '463.061',
 'BA': '',
 'BAB': '8592.0',
 'BARC': '1.23',
 'BATS': '6030.2',
 'BDEV': '',
 'BG': '157.30100000000002',
 'BLND': '42.613',
 'BLT': '261.32',
 'BNZL': '',
 'BP': '89.0',
 'BRBY': '328.0',
 'BT-A': '',
 'CCH': '310.0',
 'CCL': '',
 'CNA': '',
 'CPG': '',
 'CPI': '',
 'CRH': '172.003',
 'DC': '',
 'DGE': '',
 'DLG': '680.2130000000001',
 'EXPN': '',
 'EZJ': '',
 'FRES': '',
 'GFS': '',
 'GKN': '',
 'GLEN': '1040.4660000000001',
 'GSK': '',
 'HIK': '3428.0',
 'HL': '',
 'HMSO': '',
 'HSBA': '475.728',
 'IAG': '37.907',
 'IHG': '',
 'III': '',
 'IMT': '',
 'INTU': '',
 'ITRK': '',
 'ITV': '',
 'JMAT': '',
 'KGF': '272.798',
 'LAND': '487.98900000000003',
 'LGEN': '466.551',
 'LLOY': '422.1',
 'LSE': '',
 'MGGT': '',
 'MKS': '59.863',
 'MNDI': '190.668',
 'MRW': '',
 'NG': '185.0',
 'NXT': '',
 'OML': '49.0',
 'PRU': '',
 'PSN': '202.5

How to write `add_dates` method
http://stackoverflow.com/questions/3240458/how-to-increment-the-day-in-datetime-python

In [ ]:
for ticker in tickers_list:
    
def add_dates(time_from, time_to, dates_dict):
    return complete_dates_dict
    
def find_start_end_date(ticker):
    return start, end # dates in datetime format

def interpolate(dates_dict, ticker, start_date, end_date):
    return dates_dict_interpolated
    